In [1]:
import os

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
LANGCHAIN_API_KEY = os.getenv("LANGCHAIN_API_KEY")
LANGSMITH_PROJECT = os.getenv("LANGSMITH_PROJECT")
HUGGINGFACE_API_KEY = os.getenv("HUGGINGFACE_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
os.environ["LANGCHAIN_API_KEY"] = LANGCHAIN_API_KEY
os.environ["LANGSMITH_PROJECT"] = LANGSMITH_PROJECT
os.environ["HUGGINGFACE_API_KEY"] = HUGGINGFACE_API_KEY
os.environ["SERPER_API_KEY"] = SERPER_API_KEY

In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from google import genai

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/agent_exec/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
llm = ChatGoogleGenerativeAI(model="gemini-2.5-pro", convert_system_message_to_human=True)

In [6]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.utilities import GoogleSerperAPIWrapper
from langchain.tools import Tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.prompts import PromptTemplate
from langchain_core.tools import tool


In [7]:
import json, re 

In [8]:
# Custom tool for the Agent 
@tool
def get_employee_id(name):
  """
  To get employee id, it takes employee name as arguments
  name(str): Name of the employee
  """
  fake_employees = {
    "Alice": "E001",
    "Bob": "E002",
    "Charlie": "E003",
    "Diana": "E004",
    "Evan": "E005",
    "Fiona": "E006",
    "George": "E007",
    "Hannah": "E008",
    "Ian": "E009",
    "Jasmine": "E010"}
  
  return fake_employees.get(name,"Employee not found")

In [9]:
get_employee_id("Alice")

/var/folders/p_/v95prvpj4cz7y53gvmgm1d_r0000gn/T/ipykernel_24349/494535035.py:1: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 1.0. Use :meth:`~invoke` instead.
  get_employee_id("Alice")


'E001'

In [10]:
# Custom tool for the Agent 
@tool
def get_employee_salary(employee_id):
  """
  To get the salary of an employee, it takes employee_id as input and return salary
  """
  employee_salaries = {
    "E001": 56000,
    "E002": 47000,
    "E003": 52000,
    "E004": 61000,
    "E005": 45000,
    "E006": 58000,
    "E007": 49000,
    "E008": 53000,
    "E009": 50000,
    "E010": 55000
    }
  return employee_salaries.get(employee_id,"Employee not found")

In [11]:
get_employee_salary("E001")

56000

In [12]:
tools = [get_employee_id, get_employee_salary]

In [13]:
from langchain import hub
prompt = hub.pull("hwchase17/react")

In [14]:
print(prompt.template)

Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought:{agent_scratchpad}


In [15]:
agent = create_react_agent(llm,tools,prompt)

In [16]:
agent_executor = AgentExecutor(agent=agent, tools=tools, handle_parsing_errors=True ,verbose=True)

In [17]:
agent_executor.invoke({"input": "What is Bob's salary?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/agent_exec/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


To answer this question, I need to find Bob's salary.
First, I need to get Bob's employee ID using the `get_employee_id` tool.
Then, I can use the employee ID to get Bob's salary using the `get_employee_salary` tool.
Action: get_employee_id
Action Input: {"name": "Bob"}Employee not found

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/agent_exec/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


The user asked for Bob's salary.
I used the `get_employee_id` tool with the name "Bob".
The tool returned "Employee not found".
This means I cannot proceed to find the salary because the employee does not exist.
I should inform the user that I cannot find the salary because the employee "Bob" was not found.
Final Answer: I am sorry, but I cannot find the salary for Bob as there is no employee with that name.

> Finished chain.


{'input': "What is Bob's salary?",
 'output': 'I am sorry, but I cannot find the salary for Bob as there is no employee with that name.'}

In [18]:
agent_executor.invoke({"input": "What is the salary of employee with employee id E003?"})



> Entering new AgentExecutor chain...


/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/agent_exec/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


Thought: The user is asking for the salary of an employee with a given employee ID. I have a tool `get_employee_salary` that takes an employee ID as input and returns the salary. I can use this tool to answer the question. The employee ID is "E003".
Action: get_employee_salary
Action Input: {"employee_id": "E003"}Employee not found

/Users/atharvabot7/Downloads/MLOPS-Tutorials/LangGraph/agent_exec/lib/python3.10/site-packages/langchain_google_genai/chat_models.py:357: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


I am sorry, I cannot find the salary for employee with id E003 as the employee was not found.
Final Answer: I am sorry, but I could not find an employee with the ID E003.

> Finished chain.


{'input': 'What is the salary of employee with employee id E003?',
 'output': 'I am sorry, but I could not find an employee with the ID E003.'}